# deep-audiobook-tuner
A system that generates an apt, emotionally pertinent, unique musical score for an audiobook automatically based on the current narrative for the purpose of ameliorating user-experience while being accurate, cost-efficient, and time saving.

### Dependencies

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../"))
import tensorflow as tf
from deepaudiobooktuner.deep_audiobook_tuner import *

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Create an instance of the deepAudiobookTuner Class

In [3]:
dat = deepAudiobookTuner(audiobook_path = r'C:\Users\Danny\Downloads\grimfairytalegoldenapple.mp3')


Creating temporary directory.
----Temporary directory created.

Loading assets.


C:\Users\Danny\miniconda3\envs\deepaudiobooktuner\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Danny\miniconda3\envs\deepaudiobooktuner\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


----Loaded audio model assets. Time taken: 2.4 s
----Loaded text model assets. Time taken: 8.6 s
----Loaded music model assets. Time taken: 4.6 s
----Setup IBM transcription service. Time taken: 0.0 s

Converting mp3 to wav
----Converted to wav. Time taken: 1.2 s

Segmenting audiobook
----Segmented audio file. Time taken: 24.6 s


### Performing Sentiment analysis by invoking the analyzeSentiments() function

In [7]:
dat.analyzeSentiments()



Performing sentiment analysis

Processing clip 1:
----Text sentiment analysis
--------Transcription complete. Time taken: 23.2 s
--------Text analysis complete. Time taken: 7.6 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.6 s
----Predicting final emotion
----Clip 1 processed. Time taken: 32.4 s

Processing clip 2:
----Text sentiment analysis
--------Transcription complete. Time taken: 23.3 s
--------Text analysis complete. Time taken: 0.4 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.6 s
----Predicting final emotion
----Clip 2 processed. Time taken: 25.3 s

Processing clip 3:
----Text sentiment analysis
--------Transcription complete. Time taken: 20.8 s
--------Text analysis complete. Time taken: 0.3 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.5 s
----Predicting final emotion
----Clip 3 processed. Time taken: 22.7 s

Processing clip 4:
----Text sentiment analysis
--------Transcription c

C:\Users\Danny\miniconda3\envs\deepaudiobooktuner\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


--------Audio analysis complete. Time taken: 0.7 s
----Predicting final emotion
----Clip 17 processed. Time taken: 6.8 s

Processing clip 18:
----Text sentiment analysis
--------Transcription complete. Time taken: 24.6 s
--------Text analysis complete. Time taken: 0.3 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.7 s
----Predicting final emotion
----Clip 18 processed. Time taken: 26.7 s

Processing clip 19:
----Text sentiment analysis
--------Transcription complete. Time taken: 24.0 s
--------Text analysis complete. Time taken: 0.4 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.8 s
----Predicting final emotion
----Clip 19 processed. Time taken: 26.2 s

Processing clip 20:
----Text sentiment analysis
--------Transcription complete. Time taken: 22.1 s
--------Text analysis complete. Time taken: 0.3 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.7 s
----Predicting final emotion
----Clip 20 proces

([array(['sad'], dtype=object),
  array(['angry'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['angry'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object),
  array(['sad'], dtype=object)],
 ["Grimm's fairy tales by Jacob and Wilhelm Grimm. the golden bird. a certain king had a beautiful garden and in the garden store a tree which bore golden apples these apples were always counted and about the time when they beg

### Generating Music by invoking the generateMusic() function

In [4]:
dat.generateMusic()



Generating music


----generated Angry clip. Time taken: 15.1 s


----generated Happy clip. Time taken: 13.7 s


----generated Neutral clip. Time taken: 16.3 s


----generated Sad clip. Time taken: 14.2 s


#### Generated music clips

In [8]:
for song_emotion in dat.songs:
    print(f'{song_emotion} Music Clip:')
    dat.songs[song_emotion].stream.show('midi')

Angry Music Clip:


Happy Music Clip:


Neutral Music Clip:


Sad Music Clip:
